In [1]:
%reload_ext autoreload
%autoreload 2


In [2]:
import sys
sys.path.insert(0, r'/Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/build')

import numpy as np

import cpp_probabilities

## Load data

In [3]:
from result_saver import SaverProvider
provider = SaverProvider()
backend = provider.get_backend('ibmq_mumbai') 
job = provider.retrieve_job("cmyhbrqrmwhg008bs4h0") # Mumbai job
# job = provider.retrieve_job("cn6g47862r90008810pg") # Sherbrooke job
memory = job.result().get_memory()

print(memory.shape)


(1000, 9)


In [4]:
from soft_info import get_KDEs, get_repcode_layout, get_repcode_IQ_map

## Mumbai
layout = [25, 19, 26, 22, 16] # To implement: into scratch job_data
kde_dict, scaler_dict = get_KDEs(provider, tobecalib_job="cmyhbrqrmwhg008bs4h0")
# kde_dict, scaler_dict = get_KDEs(provider, device='ibmq_mumbai', qubits=list(range(27)), bandwidths=0.2, plot=False)
synd_rounds = 3

## Sherbrooke
# layout = get_repcode_layout(30, provider.get_backend('ibm_sherbrooke') )
# kde_dict, scaler_dict = get_KDEs(provider, 'ibm_sdherbrooke', list(range(127)), bandwidths=0.2, plot=False)
# synd_rounds = 35

print("Generating the qubit mapping...")
qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes

Generating the qubit mapping...


# Metadata structure

In [21]:
import json
import pandas as pd
import os
from typing import Dict, List, Optional, Union
from sklearn.preprocessing import StandardScaler

from Scratch import find_and_create_scratch

import numpy as np

def generate_kde_grid(kde_dict, num_points, num_std_dev=3):
    grid_dict = {}

    # Define the grid range and create grid points
    grid_range_real = np.linspace(-num_std_dev, num_std_dev, num_points)
    grid_range_imag = np.linspace(-num_std_dev, num_std_dev, num_points)
    grid_x, grid_y = np.meshgrid(grid_range_real, grid_range_imag)
    grid_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T

    for qubit_idx, (kde_0, kde_1) in kde_dict.items():
        # Evaluate KDE on the grid for both states
        grid_density_0 = (kde_0.score_samples(grid_points)).reshape(grid_x.shape)
        grid_density_1 = (kde_1.score_samples(grid_points)).reshape(grid_x.shape)
        
        # Create an instance of GridData and store in dictionary
        # grid_dict[qubit_idx] = cpp_probabilities.GridData(grid_x, grid_y, grid_density_0, grid_density_1)
        grid_dict[qubit_idx] = (grid_x, grid_y, grid_density_0, grid_density_1)

    return grid_dict



def update_grid_metadata(metadata_path, job_id, creation_date, grid_file_path):
    """
    Update or create grid metadata.

    Args:
        metadata_path (str): Path to the metadata file.
        job_id (str): The job ID associated with the grid.
        creation_date (str): The creation date of the job.
        grid_file_path (str): Path to the grid file.

    """
    # Check if the metadata file exists
    if os.path.exists(metadata_path):
        # Read existing metadata
        with open(metadata_path, 'r') as file:
            metadata = json.load(file)
    else:
        # Initialize metadata if file does not exist
        metadata = {}

    # Update or create the metadata entry
    metadata[job_id] = {
        'creation_date': creation_date,
        'grid_file_path': grid_file_path
    }

    # Write the updated metadata back to the file
    with open(metadata_path, 'w') as file:
        json.dump(metadata, file, indent=4)



def save_grid(grid_dict, filename):
    """
    Save a grid dictionary as a JSON file.

    Args:
        grid_dict (dict): Dictionary containing tuples of NumPy arrays.
        filename (str): The name of the file to save the grid data.
    """
    serializable_grid_dict = {}
    for qubit_idx, (grid_x, grid_y, grid_density_0, grid_density_1) in grid_dict.items():
        serializable_grid_dict[qubit_idx] = {
            'grid_x': grid_x.tolist(), 
            'grid_y': grid_y.tolist(), 
            'grid_density_0': grid_density_0.tolist(), 
            'grid_density_1': grid_density_1.tolist()
        }

    with open(filename, 'w') as file:
        json.dump(serializable_grid_dict, file, indent=4)

def load_grid(filename):
    """
    Load a grid dictionary from a JSON file and create GridData objects.

    Args:
        filename (str): The name of the file to load the grid data from.

    Returns:
        dict: Dictionary containing GridData objects.
    """
    with open(filename, 'r') as file:
        serializable_grid_dict = json.load(file)

    grid_dict = {}
    for qubit_idx, grid_data in serializable_grid_dict.items():
        grid_x = np.array(grid_data['grid_x'])
        grid_y = np.array(grid_data['grid_y'])
        grid_density_0 = np.array(grid_data['grid_density_0'])
        grid_density_1 = np.array(grid_data['grid_density_1'])

        # Create GridData objects
        grid_dict[int(qubit_idx)] = cpp_probabilities.GridData(
            grid_x, grid_y, grid_density_0, grid_density_1
        )

    return grid_dict



# Example usage
root_dir = find_and_create_scratch()
grid_dict = generate_kde_grid(kde_dict, 10, num_std_dev=3)  # Replace with actual kde_dict
save_grid(grid_dict, root_dir + '/calibration_grids/your_grid_file.json')
grid = load_grid(root_dir + '/calibration_grids/your_grid_file.json')
print(grid[0])



In [ ]:
import os
from cpp import process_scaler_dict

def create_or_load_kde_grid(provider, tobecalib_job, num_points, num_std_dev=3):
    """
    Create or load a KDE grid. If the grid for the specified job is already generated and saved, it loads the grid.
    Otherwise, it creates a new grid, saves it, and updates the metadata.

    Args:
        provider: Quantum provider for obtaining calibration data.
        tobecalib_job (str): The job ID to be calibrated.
        grid_folder (str): Folder where grids are saved.
        metadata_file (str): Path to the grid metadata file.
        num_points (int): Number of points in the grid.
        num_std_dev (float): Number of standard deviations for grid range.

    Returns:
        dict: Dictionary containing GridData objects.
    """
    root_dir = find_and_create_scratch()
    grid_folder = root_dir + '/calibration_grids'
    metadata_file = root_dir + '/calibration_grid_metadata.json'

    if os.path.exists(metadata_file):
        with open(metadata_file, 'r') as file:
            grid_metadata = json.load(file)
    else:
        grid_metadata = {}

    grid_entry = grid_metadata.get(tobecalib_job) #TODO find the closest job id func

    kde_dict, scaler_dict = get_KDEs(provider, tobecalib_job=tobecalib_job)
    if grid_entry:
        # Load the existing grid
        grid_file_path = grid_entry['grid_file_path']
        return load_grid(grid_file_path)
    else:
        # Generate a new grid        
        grid_dict = generate_kde_grid(kde_dict, num_points, num_std_dev)

        # Save the new grid
        grid_file_path = os.path.join(grid_folder, f"{tobecalib_job}_grid.json")
        save_grid(grid_dict, grid_file_path)

        # Update metadata
        update_grid_metadata(metadata_file, tobecalib_job, grid_file_path=grid_file_path)

        return grid_dict, process_scaler_dict(scaler_dict)

# Example usage
grid_dict = create_or_load_kde_grid(provider, 'cmyhbrqrmwhg008bs4h0', '/path/to/calibration_grids', '/path/to/calibration_grid_metadata.json', 10, num_std_dev=3)


# Generate grid data

In [24]:
import numpy as np

def generate_kde_grid(kde_dict, num_points, num_std_dev=10):
    grid_dict = {}

    # Define the grid range and create grid points
    grid_range_real = np.linspace(-num_std_dev, num_std_dev, num_points)
    grid_range_imag = np.linspace(-num_std_dev, num_std_dev, num_points)
    grid_x, grid_y = np.meshgrid(grid_range_real, grid_range_imag)
    grid_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T

    for qubit_idx, (kde_0, kde_1) in kde_dict.items():
        # Evaluate KDE on the grid for both states
        grid_density_0 = (kde_0.score_samples(grid_points)).reshape(grid_x.shape)
        grid_density_1 = (kde_1.score_samples(grid_points)).reshape(grid_x.shape)
        
        # Create an instance of GridData and store in dictionary
        grid_dict[qubit_idx] = cpp_probabilities.GridData(grid_x, grid_y, grid_density_0, grid_density_1)

    return grid_dict

grid_dict = generate_kde_grid(kde_dict, 10, num_std_dev=3)
print(len(grid_dict))


27


# Test grid data

In [25]:
from cpp import process_scaler_dict


processed_scaler_dict = process_scaler_dict(scaler_dict)

counts = cpp_probabilities.get_counts(memory, qubit_mapping, grid_dict, processed_scaler_dict, synd_rounds)
(sorted(counts.items(), key=lambda x: x[1], reverse=True))

[('000 00 00 00', 866),
 ('000 01 00 00', 15),
 ('000 10 10 10', 11),
 ('000 01 01 01', 10),
 ('000 00 01 00', 9),
 ('000 10 00 00', 9),
 ('100 00 10 00', 8),
 ('100 10 00 10', 7),
 ('000 10 10 00', 6),
 ('001 00 00 00', 5),
 ('010 11 00 11', 5),
 ('000 00 10 00', 4),
 ('001 00 01 00', 4),
 ('001 01 00 00', 4),
 ('001 01 00 01', 4),
 ('100 10 00 00', 4),
 ('100 00 00 00', 3),
 ('000 00 10 10', 2),
 ('000 01 01 00', 2),
 ('000 10 00 10', 2),
 ('000 00 00 01', 1),
 ('000 00 00 10', 1),
 ('000 01 00 01', 1),
 ('000 10 11 10', 1),
 ('000 11 01 01', 1),
 ('001 01 01 00', 1),
 ('001 01 01 01', 1),
 ('001 01 10 00', 1),
 ('001 10 10 00', 1),
 ('001 11 10 10', 1),
 ('010 00 00 00', 1),
 ('010 00 11 00', 1),
 ('010 01 00 11', 1),
 ('010 01 10 01', 1),
 ('010 10 00 00', 1),
 ('010 10 01 00', 1),
 ('010 11 00 00', 1),
 ('010 11 01 00', 1),
 ('100 00 10 10', 1),
 ('100 10 10 00', 1)]